<a href="https://colab.research.google.com/github/Raymond-Owen1-137/BMRB-DATA-COLLECTOR1.0/blob/main/bmrb_parser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def fetch_pdb_file(pdb_id):
    """
    Downloads the PDB file for the given PDB ID.
    Returns the file content as a string or None if there's an error.
    """
    url = f"https://files.rcsb.org/download/{pdb_id}.pdb"
    try:
        response = requests.get(url)
        if response.status_code != 200:
            print(f"❌ Failed to fetch PDB file for {pdb_id}")
            return None
        # Light sleep to avoid spamming the server
        time.sleep(0.5)
        return response.text

    except requests.exceptions.RequestException as e:
        print(f"❌ Error fetching PDB file for {pdb_id}: {e}")
        return None

def fetch_avs_file(bmrb_id):
    """
    Fetches the AVS_full.txt file content for a given BMRB ID.
    Returns the text content or None if there's an error.
    """
    avs_url = f"https://bmrb.io/ftp/pub/bmrb/entry_directories/bmr{bmrb_id}/validation/AVS_full.txt"
    try:
        response = requests.get(avs_url)
        if response.status_code != 200:
            return None
        return response.text
    except requests.exceptions.RequestException as e:
        print(f"❌ Error fetching AVS file for BMRB {bmrb_id}: {e}")
        return None

# ---------------------------------------------------------------------------------
# Parsing Functions
# ---------------------------------------------------------------------------------

def parse_pdb_secondary_structure(pdb_content):
    """
    Parses PDB content to extract secondary structure info.
    Returns (helices, sheets) where each is a list of (chain, start_res, end_res).
    """
    helices, sheets = [], []
    if not pdb_content:
        return helices, sheets

    for line in pdb_content.splitlines():
        if line.startswith("HELIX"):
            # Chain is at column 20, start_res is columns 22-25, end_res is 34-37
            chain = line[19]
            start_res = int(line[21:25].strip())
            end_res = int(line[33:37].strip())
            helices.append((chain, start_res, end_res))

        elif line.startswith("SHEET"):
            # Chain is at column 22, start_res is columns 23-26, end_res is 34-37
            chain = line[21]
            start_res = int(line[22:26].strip())
            end_res = int(line[33:37].strip())
            sheets.append((chain, start_res, end_res))

    return helices, sheets

def assign_secondary_structure(res_num, chain, helices, sheets):
    """
    Assigns secondary structure based on chain and residue number.
    Returns one of: "Helix", "Sheet", or "Coil".
    """
    # Check if the residue is in a helix
    for helix_chain, start_res, end_res in helices:
        if helix_chain == chain and start_res <= res_num <= end_res:
            return "Helix"

    # Check if the residue is in a beta sheet
    for sheet_chain, start_res, end_res in sheets:
        if sheet_chain == chain and start_res <= res_num <= end_res:
            return "Sheet"

    # Default to Coil
    return "Coil"

def parse_residue_data(file_content):
    """
    Extracts residue chemical shifts from AVS_full.txt.
    Returns a list of dicts, each with keys: 'Residue', 'C', 'CA', 'CB'.
    """
    if not file_content:
        return []

    # Regex finds lines: (Residue) then "Ave C Shift Values>>" and captures the entire shift section
    residue_blocks = re.findall(r"(\w\d+).*?Ave C Shift Values>>\s*(.*?)\n\n", file_content, re.DOTALL)
    parsed_data = []

    for res_id, shift_section in residue_blocks:
        residue_data = {"Residue": res_id, "C": "N/A", "CA": "N/A", "CB": "N/A"}

        c_match = re.search(r"C\s*::\s*([\-\d.]+)\s*\([\d.]+\)", shift_section)
        ca_match = re.search(r"CA\s*::\s*([\-\d.]+)\s*\([\d.]+\)", shift_section)
        cb_match = re.search(r"CB\s*::\s*([\-\d.]+)\s*\([\d.]+\)", shift_section)

        if c_match:
            residue_data["C"] = c_match.group(1)
        if ca_match:
            residue_data["CA"] = ca_match.group(1)
        if cb_match:
            residue_data["CB"] = cb_match.group(1)

        parsed_data.append(residue_data)

    return parsed_data

# ---------------------------------------------------------------------------------
# Main Processing Workflow
# ---------------------------------------------------------------------------------

import os
import requests
import re
import csv
import time
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
from bs4 import BeautifulSoup

# ---------------------------------------------------------------------------------
# Constants
# ---------------------------------------------------------------------------------

QUERY_GRID_URL = (
    "https://bmrb.io/search/query_grid/?data_types%5B%5D=carbon_shifts"
    "&polymers%5B%5D=polypeptide%28L%29&polymer_join_type=AND"
)

# Dictionary to convert 1-letter to 3-letter amino acid names
AMINO_ACIDS = {
    "A": "ALA", "R": "ARG", "N": "ASN", "D": "ASP", "C": "CYS",
    "E": "GLU", "Q": "GLN", "G": "GLY", "H": "HIS", "I": "ILE",
    "L": "LEU", "K": "LYS", "M": "MET", "F": "PHE", "P": "PRO",
    "S": "SER", "T": "THR", "W": "TRP", "Y": "TYR", "V": "VAL"
}

# Create base data directory
DATA_FOLDER = "data"
os.makedirs(DATA_FOLDER, exist_ok=True)

# Output folders inside the "data/" subdirectory
FOLDERS = {
    "Helix": os.path.join(DATA_FOLDER, "alpha_chemicalshift"),
    "Sheet": os.path.join(DATA_FOLDER, "beta_chemicalshift"),
    "Coil": os.path.join(DATA_FOLDER, "coil_chemicalshift")
}

# Ensure output folders exist inside "data/"
for folder in FOLDERS.values():
    os.makedirs(folder, exist_ok=True)

# Log file inside "data/"
LOG_FILE = os.path.join(DATA_FOLDER, "log.txt")

# ---------------------------------------------------------------------------------
# Main Processing Workflow
# ---------------------------------------------------------------------------------

def process_bmrb_entries():
    """
    Fetch BMRB IDs, retrieve PDB files, parse chemical shifts, and save to CSV.
    All files will be stored inside the "data/" directory.
    """
    print("🔍 Fetching BMRB IDs...")
    bmrb_ids = fetch_bmrb_ids(n=10)
    if not bmrb_ids:
        print("No BMRB IDs found. Exiting.")
        return

    print("🔍 Fetching PDB IDs in parallel...")
    with ThreadPoolExecutor() as executor:
        pdb_ids = list(tqdm(executor.map(fetch_pdb_id, bmrb_ids), total=len(bmrb_ids)))

    for bmrb_id, pdb_id in zip(bmrb_ids, pdb_ids):
        if not pdb_id:
            with open(LOG_FILE, "a") as log:
                log.write(f"BMRB {bmrb_id} missing PDB entry.\n")
            continue

        pdb_content = fetch_pdb_file(pdb_id)
        if not pdb_content:
            continue

        helices, sheets = parse_pdb_secondary_structure(pdb_content)
        file_content = fetch_avs_file(bmrb_id)

        if not file_content:
            with open(LOG_FILE, "a") as log:
                log.write(f"BMRB {bmrb_id} missing AVS file.\n")
            continue

        residue_data = parse_residue_data(file_content)

        for res in residue_data:
            chain, res_num = parse_chain_and_resnum(res["Residue"])
            one_letter_code = res["Residue"][0]
            three_letter = AMINO_ACIDS.get(one_letter_code.upper(), "UNK")

            sec_struct = assign_secondary_structure(res_num, chain, helices, sheets)
            folder = FOLDERS[sec_struct]
            sec_char = {"Helix": "a", "Sheet": "b", "Coil": "c"}[sec_struct]

            # ✅ Fix file path issue
            filename = os.path.join(folder, f"{three_letter}_{sec_struct}.csv")

            with open(filename, "a", newline="") as csvfile:
                writer = csv.writer(csvfile)
                if os.path.getsize(filename) == 0:
                    writer.writerow(["Residue", "C", "CA", "CB", "Secondary_Structure"])
                writer.writerow([res["Residue"], res["C"], res["CA"], res["CB"], sec_char])


        print(f"✅ Processed BMRB {bmrb_id} (PDB: {pdb_id})")


if __name__ == "__main__":
    process_bmrb_entries()


In [ ]:
import os
import pandas as pd

# Define base data folder
DATA_FOLDER = "data"

# Define input subfolders
FOLDERS = {
    "Helix": os.path.join(DATA_FOLDER, "alpha_chemicalshift"),
    "Sheet": os.path.join(DATA_FOLDER, "beta_chemicalshift"),
    "Coil": os.path.join(DATA_FOLDER, "coil_chemicalshift")
}

# Output file for the combined dataset
OUTPUT_FILE = os.path.join(DATA_FOLDER, "combined_chemical_shifts.csv")

# Define column names for the dataset
columns = ["Residue_Type", "Residue_ID", "C", "CA", "CB", "Secondary_Structure"]

# List to store extracted data
data = []

def extract_data_from_csv(folder_name, secondary_structure):
    """
    Reads all CSV files from a given folder, extracts chemical shift data,
    and appends it to the global data list.

    Args:
        folder_name (str): The folder containing CSV files.
        secondary_structure (str): The secondary structure type (Helix, Sheet, Coil).
    """
    for filename in os.listdir(folder_name):
        if filename.endswith(".csv"):  # Process only CSV files
            file_path = os.path.join(folder_name, filename)

            # Extract residue type from the filename (e.g., "ALA_helix.csv" → "ALA")
            residue_type = filename.split("_")[0]

            # Read the CSV file
            df = pd.read_csv(file_path)

            # Ensure required columns exist
            if {"Residue", "C", "CA", "CB"}.issubset(df.columns):
                for _, row in df.iterrows():
                    # Extract residue ID (e.g., "A10" → Residue Type: "A", ID: "10")
                    residue_id = row["Residue"]

                    # Append extracted data to list
                    data.append([
                        residue_type,  # Residue type (e.g., ALA, LYS)
                        residue_id,    # Residue ID (e.g., A10)
                        row["C"],      # Carbonyl shift
                        row["CA"],     # Alpha Carbon shift
                        row["CB"],     # Beta Carbon shift
                        secondary_structure  # Secondary Structure (Helix, Sheet, Coil)
                    ])

# Extract data from all three structure types
for struct_type, folder in FOLDERS.items():
    extract_data_from_csv(folder, struct_type)

# Convert data list to a DataFrame
df_combined = pd.DataFrame(data, columns=columns)

# Handle missing values (replace "N/A" with NaN for easier processing)
df_combined.replace("N/A", pd.NA, inplace=True)

# Save to CSV for further analysis
df_combined.to_csv(OUTPUT_FILE, index=False)

print(f"✅ Data extraction complete! Combined dataset saved as '{OUTPUT_FILE}'.")
